In [1]:
from keras.preprocessing.sequence import pad_sequences
from keras.preprocessing.text import Tokenizer
from sklearn.model_selection import train_test_split
import pandas as pd
import numpy as np
import re
import nltk
from nltk.corpus import stopwords

from numpy import array
from keras.preprocessing.text import one_hot
from keras.preprocessing.sequence import pad_sequences
from keras.models import Sequential
from keras.layers.core import Activation, Dropout, Dense
from keras.layers import Flatten, LSTM
from keras.layers import GlobalMaxPooling1D
from keras.models import Model
from keras.layers.embeddings import Embedding
from sklearn.model_selection import train_test_split
from keras.preprocessing.text import Tokenizer
from keras.layers import Input
from keras.layers.merge import Concatenate



Using TensorFlow backend.
D:\Users\kpannegatla\AppData\Local\Continuum\anaconda3\envs\tf\lib\site-packages\tensorflow\python\framework\dtypes.py:516: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
D:\Users\kpannegatla\AppData\Local\Continuum\anaconda3\envs\tf\lib\site-packages\tensorflow\python\framework\dtypes.py:517: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
D:\Users\kpannegatla\AppData\Local\Continuum\anaconda3\envs\tf\lib\site-packages\tensorflow\python\framework\dtypes.py:518: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np

In [2]:
data = pd.read_csv('training_variants')
print("Dimentions : ",data.shape)
print("Feature names: ", data.columns.values)
print("\n",data.head())

Dimentions :  (3321, 4)
Feature names:  ['ID' 'Gene' 'Variation' 'Class']

    ID    Gene             Variation  Class
0   0  FAM58A  Truncating Mutations      1
1   1     CBL                 W802*      2
2   2     CBL                 Q249E      2
3   3     CBL                 N454D      3
4   4     CBL                 L399V      4


In [3]:
data_text = pd.read_csv('training_text', sep='\|\|', engine='python', names=['ID','TEXT'], skiprows=1)
print("Dimentions : ",data_text.shape)
print("Feature names: ", data_text.columns.values)
print("\n",data_text.head())

Dimentions :  (3321, 2)
Feature names:  ['ID' 'TEXT']

    ID                                               TEXT
0   0  Cyclin-dependent kinases (CDKs) regulate a var...
1   1   Abstract Background  Non-small cell lung canc...
2   2   Abstract Background  Non-small cell lung canc...
3   3  Recent evidence has demonstrated that acquired...
4   4  Oncogenic mutations in the monomeric Casitas B...


In [165]:
#loading stop words
stop_words = set(stopwords.words('english'))


def nlp_preprocessing(total_text, index, column):
    if type(total_text) is not int:
        string = ''
        # replacing all special characters with space
        total_text = re.sub('[^a-zA-Z0-9\n]',' ',total_text)
        #removing all numbers
        total_text = re.sub(r'[0-9]+', '', total_text)
        # removing all extra spaces
        total_text = re.sub('\s+',' ',total_text)
        # converting all chars to lower
        total_text = total_text.lower()
        

        for word in total_text.split():
            #removing stop words
            if not word in stop_words:
                string += word+' '
                
        data_text[column][index] = string

In [166]:
for index, row in data_text.iterrows():
    if type(row['TEXT']) is str:
        nlp_preprocessing(row["TEXT"], index, column = 'TEXT')
    else:
        print("There is no text for some Indexes: ", index)
        

D:\Users\kpannegatla\AppData\Local\Continuum\anaconda3\envs\tf\lib\site-packages\ipykernel_launcher.py:23: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


There is no text for some Indexes:  1109
There is no text for some Indexes:  1277
There is no text for some Indexes:  1407
There is no text for some Indexes:  1639
There is no text for some Indexes:  2755


In [167]:
data = pd.merge(data, data_text, on="ID", how='left')
data.head()

,ID,Gene,Variation,Class,TEXT
0,0,FAM58A,Truncating Mutations,1,cyclin dependent kinases cdks regulate variety...
1,1,CBL,W802*,2,abstract background non small cell lung cancer...
2,2,CBL,Q249E,2,abstract background non small cell lung cancer...
3,3,CBL,N454D,3,recent evidence demonstrated acquired uniparen...
4,4,CBL,L399V,4,oncogenic mutations monomeric casitas b lineag...


In [173]:
#data.to_csv (r'D:/Users/kpannegatla/Desktop/ML/Cancer/data.csv', index = False, header=True)

In [168]:
data.loc[data['TEXT'].isnull(),'TEXT']

1109    NaN
1277    NaN
1407    NaN
1639    NaN
2755    NaN
Name: TEXT, dtype: object

In [169]:
data['TEXT'].fillna(data['Gene']+' '+data['Variation'], inplace=True)

In [170]:
data.head()

,ID,Gene,Variation,Class,TEXT
0,0,FAM58A,Truncating Mutations,1,cyclin dependent kinases cdks regulate variety...
1,1,CBL,W802*,2,abstract background non small cell lung cancer...
2,2,CBL,Q249E,2,abstract background non small cell lung cancer...
3,3,CBL,N454D,3,recent evidence demonstrated acquired uniparen...
4,4,CBL,L399V,4,oncogenic mutations monomeric casitas b lineag...


In [64]:
from nltk import word_tokenize
tokens = [word_tokenize(sen) for sen in data.TEXT]

In [65]:
data['tokens'] = tokens

In [66]:
data.head()

,ID,Gene,Variation,Class,TEXT,tokens
0,0,FAM58A,Truncating Mutations,1,cyclin dependent kinases cdks regulate variety...,"[cyclin, dependent, kinases, cdks, regulate, v..."
1,1,CBL,W802*,2,abstract background non small cell lung cancer...,"[abstract, background, non, small, cell, lung,..."
2,2,CBL,Q249E,2,abstract background non small cell lung cancer...,"[abstract, background, non, small, cell, lung,..."
3,3,CBL,N454D,3,recent evidence demonstrated acquired uniparen...,"[recent, evidence, demonstrated, acquired, uni..."
4,4,CBL,L399V,4,oncogenic mutations monomeric casitas b lineag...,"[oncogenic, mutations, monomeric, casitas, b, ..."


## Getting total words and length of sentences

In [67]:
data_train, data_test = train_test_split(data, 
                                         test_size=0.20, 
                                         random_state=42)

In [68]:
all_training_words = [word for tokens in data_train["tokens"] for word in tokens]
training_sentence_lengths = [len(tokens) for tokens in data_train["tokens"]]
TRAINING_VOCAB = sorted(list(set(all_training_words)))
print("%s words total, with a vocabulary size of %s" % (len(all_training_words), len(TRAINING_VOCAB)))
print("Max sentence length is %s" % max(training_sentence_lengths))

16106085 words total, with a vocabulary size of 85159
Max sentence length is 50880


In [69]:
import statistics
statistics.mean(training_sentence_lengths) 

6064.038027108434

In [102]:
all_test_words = [word for tokens in data_test['tokens'] for word in tokens]
test_sentence_lengths = [len(tokens) for tokens in data_test['tokens']]
TEST_VOCAB = sorted(list(set(all_test_words)))
print('%s words total, with a vocabulary size of %s' % (len(all_test_words), len(TEST_VOCAB)))
print('Max sentence length is %s' % max(test_sentence_lengths))

4254511 words total, with a vocabulary size of 48102
Max sentence length is 31714


In [103]:
import statistics
statistics.mean(test_sentence_lengths) 

6397.760902255639

In [89]:
data_train.head()

,ID,Gene,Variation,Class,TEXT,tokens
2241,2241,PTEN,D331G,4,tumor suppressor gene pten frequently mutated ...,"[tumor, suppressor, gene, pten, frequently, mu..."
1459,1459,FGFR2,V755I,5,discovery oncogenic driver mutations subsequen...,"[discovery, oncogenic, driver, mutations, subs..."
468,468,TP53,K291Q,1,tumor suppressor p dependent apoptosis thought...,"[tumor, suppressor, p, dependent, apoptosis, t..."
3271,3271,RET,L790F,7,mutations ret receptor tyrosine kinase associa...,"[mutations, ret, receptor, tyrosine, kinase, a..."
1580,1580,MPL,W515K,2,recently gain function mpl mutation mplwl desc...,"[recently, gain, function, mpl, mutation, mplw..."


In [95]:
X = data.drop(['ID','Class','tokens'], axis=1)

y = data['Class']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.20, random_state=42)

from keras.utils import to_categorical
y_train = to_categorical(y_train)
y_test = to_categorical(y_test)

## preparing data for TEXT feature

In [101]:
X1_train = list(X_train["TEXT"])
X1_test = list(X_test["TEXT"])


In [104]:
# The maximum number of words to be used. (most frequent)
#MAX_NB_WORDS = 100000
# Max number of words in each complaint.
MAX_SEQUENCE_LENGTH = 2000
# This is fixed.
EMBEDDING_DIM = 100

In [105]:
tokenizer = Tokenizer(num_words=len(TRAINING_VOCAB))
tokenizer.fit_on_texts(X1_train)

X1_train = tokenizer.texts_to_sequences(X1_train)
X1_test = tokenizer.texts_to_sequences(X1_test)

vocab_size = len(tokenizer.word_index) + 1


X1_train = pad_sequences(X1_train, padding='post', maxlen=MAX_SEQUENCE_LENGTH)
X1_test = pad_sequences(X1_test, padding='post', maxlen=MAX_SEQUENCE_LENGTH)

## Preparing data for Gene and Variation feature

In [114]:
X2_train  = X_train[['Gene','Variation']]

X2_test  = X_test[['Gene','Variation']]

In [115]:
X2_train = pd.get_dummies(X2_train, columns=['Gene','Variation'], drop_first=True)

X2_test = pd.get_dummies(X2_test, columns=['Gene','Variation'], drop_first=True)

In [120]:
print(X2_train.shape)
#print(X1_train.shape)
print(X2_test.shape)

(2656, 2646)
(665, 769)


## Creating NN
Keras Functional API

In [139]:
MAX_SEQUENCE_LENGTH = 2000
size = X2_train.shape[1]
EMBEDDING_DIM = 100

# submodel 1

In [143]:
input_1 = Input(shape=(MAX_SEQUENCE_LENGTH,))

embedding_layer = Embedding(vocab_size, EMBEDDING_DIM)(input_1)
LSTM_Layer_1 = LSTM(64)(embedding_layer)
Dropout_1 = Dropout(0.3)(LSTM_Layer_1)


## submodel 2

In [144]:
input_2 = Input(shape=(size,)) 

dense_layer_1 = Dense(32, activation='relu')(input_2)
Dropout_2 = Dropout(0.3)(dense_layer_1)
dense_layer_2 = Dense(32, activation='relu')(Dropout_2)
Dropout_3 = Dropout(0.3)(dense_layer_2)

## Final Model

In [156]:
concat_layer = Concatenate()([Dropout_1, Dropout_3])
dense_layer_3 = Dense(32, activation='relu')(concat_layer)
output = Dense(10, activation='softmax')(dense_layer_3)
model = Model(inputs=[input_1, input_2], outputs=output)

In [157]:
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['acc'])
print(model.summary())

Model: "model_3"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_13 (InputLayer)           (None, 2646)         0                                            
__________________________________________________________________________________________________
input_12 (InputLayer)           (None, 2000)         0                                            
__________________________________________________________________________________________________
dense_7 (Dense)                 (None, 32)           84704       input_13[0][0]                   
__________________________________________________________________________________________________
embedding_10 (Embedding)        (None, 2000, 100)    8515400     input_12[0][0]                   
____________________________________________________________________________________________

In [158]:
#!pip install pydot

In [159]:
"""from keras.utils import plot_model
plot_model(model, to_file='model_plot3.png', show_shapes=True, show_layer_names=True)"""

"from keras.utils import plot_model\nplot_model(model, to_file='model_plot3.png', show_shapes=True, show_layer_names=True)"

In [160]:
history = model.fit(x=[X1_train, X2_train], y=y_train, batch_size=128, epochs=10, verbose=1, validation_split=0.2)

D:\Users\kpannegatla\AppData\Local\Continuum\anaconda3\envs\tf\lib\site-packages\tensorflow_core\python\framework\indexed_slices.py:424: UserWarning: Converting sparse IndexedSlices to a dense Tensor of unknown shape. This may consume a large amount of memory.
  "Converting sparse IndexedSlices to a dense Tensor of unknown shape. "


Train on 2124 samples, validate on 532 samples
Epoch 1/10
2124/2124 [==============================] - 584s 275ms/step - loss: 2.2525 - acc: 0.2524 - val_loss: 2.0461 - val_acc: 0.3083
Epoch 2/10
2124/2124 [==============================] - 1075s 506ms/step - loss: 1.9027 - acc: 0.2764 - val_loss: 1.8635 - val_acc: 0.3383
Epoch 3/10
2124/2124 [==============================] - 549s 258ms/step - loss: 1.8285 - acc: 0.2924 - val_loss: 1.8094 - val_acc: 0.3553
Epoch 4/10
2124/2124 [==============================] - 2023s 953ms/step - loss: 1.7216 - acc: 0.3649 - val_loss: 1.6496 - val_acc: 0.4248
Epoch 5/10
2124/2124 [==============================] - 596s 281ms/step - loss: 1.4847 - acc: 0.4750 - val_loss: 1.4708 - val_acc: 0.4850
Epoch 6/10
2124/2124 [==============================] - 895s 421ms/step - loss: 1.2367 - acc: 0.5612 - val_loss: 1.3573 - val_acc: 0.5263
Epoch 7/10
 896/2124 [===========>..................] - ETA: 5:38 - loss: 1.0183 - acc: 0.6708

ResourceExhaustedError:  OOM when allocating tensor with shape[128,64] and type float on /job:localhost/replica:0/task:0/device:CPU:0 by allocator cpu
	 [[node gradients/lstm_14/TensorArrayV2Stack/TensorListStack_grad/TensorListFromTensor (defined at D:\Users\kpannegatla\AppData\Local\Continuum\anaconda3\envs\tf\lib\site-packages\tensorflow_core\python\framework\ops.py:1751) ]]
Hint: If you want to see a list of allocated tensors when OOM happens, add report_tensor_allocations_upon_oom to RunOptions for current allocation info.
 [Op:__inference_keras_scratch_graph_6196]

Function call stack:
keras_scratch_graph
